所要時間：　プログラム実行時間とパラメータチューニングを含めて2時間です （コードのみだと30分）
参考URL：  
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html  
https://lightgbm.readthedocs.io/en/latest/Python-API.html  
https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.rst  

In [1]:
import numpy as np
import lightgbm as lgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from progressbar import ProgressBar

In [2]:
from bayes_opt import  Bayes_opt_lgb_v2

In [4]:
train = pd.read_csv("./dota2Dataset/dota2Train.csv", header=None)
test = pd.read_csv("./dota2Dataset/dota2Test.csv", header=None)
target_tr = train[0]
target_te = test[0]
target_tr[target_tr < 0] = 0
target_te[target_te < 0] = 0
train = train.drop([0], axis=1)
test = test.drop([0], axis=1)

In [5]:
for i in range(4, len(train.iloc[0,3:])):
    if len(np.unique(train[i].values)) == 1:
        train = train.drop([i], axis=1)
        test = test.drop([i], axis=1)
        print("feature ", i , "is dropped.")

feature  27 is dropped.
feature  111 is dropped.


In [6]:
for i in range(0, 3):
    train.iloc[:,i] = train.iloc[:,i].astype("category")
    test.iloc[:,i] = test.iloc[:,i].astype("category")

In [7]:
# params は探索するパラメータ
params = {
        'max_depth': [3, 10],
        'num_leaves' : [8, 64],
        'learning_rate' : [0.01, 0.1],
        'feature_fraction' : [0.6, 0.99],
        'bagging_fraction' : [0.6, 0.99],
        'bagging_freq': [1, 10],
        'min_data_in_leaf': [3, 25],
        'subsample': [0.6, 0.99],
        'colsample_bytree': [0.6, 0.99],
        'reg_lambda': [0.5, 15.0],
        'reg_alpha': [0.5, 15.0],  
}

#basic_paramsは固定パラメータ
basic_params = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary', #回帰ならobjective = regression
    'metric' : {'binary_logloss'}, #回帰ならmseなど
    #'num_class': 3, # 回帰ならコメントアウト
    'verbose' : 0,
    'num_threads':15
}

bayes_lgb = Bayes_opt_lgb_v2(opt_param = params, basic_params = basic_params, dat_size = 30000)
opt_parameter = bayes_lgb.fit(train.values, target_tr.values, ep=100, y_amp=100.0, maximize=False)


[10.   64.    0.1   0.99  0.99 10.   25.    0.99  0.99 15.   15.  ]
epoch: 0
parameter:  {'max_depth': 4, 'num_leaves': 44, 'learning_rate': 0.054513093658002776, 'feature_fraction': 0.7727758295985573, 'bagging_fraction': 0.9244454299987683, 'bagging_freq': 1, 'min_data_in_leaf': 20, 'subsample': 0.8765911238387747, 'colsample_bytree': 0.7035707894411178, 'reg_lambda': 4.322241267419809, 'reg_alpha': 2.6804790752956063, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 1, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.677432 + 0.000342553
[200]	cv_agg's binary_logloss: 0.671834 + 0.000556438
[300]	cv_agg's binary_logloss: 0.668688 + 0.000650099
[400]	cv_agg's binary_logloss: 0.666777 + 0.000726525
[500]	cv_agg's binary_logloss: 0.665408 + 0.00079037
[600]	cv_agg's binary_logloss: 0.664498 + 0.000867693
[700]	cv_agg's binary_logloss: 0.663808 + 0.000918069
[800]	cv_agg's binary_logloss: 0.66336 + 0.000970647
[900]	cv_agg's binary_logloss: 0.6

parameter:  {'max_depth': 10, 'num_leaves': 12, 'learning_rate': 0.09209802517634882, 'feature_fraction': 0.9800285489015506, 'bagging_fraction': 0.6045863130888752, 'bagging_freq': 1, 'min_data_in_leaf': 5, 'subsample': 0.6302192884594014, 'colsample_bytree': 0.9708970666572276, 'reg_lambda': 3.545064688517205, 'reg_alpha': 0.5313114100262888, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 1, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.667235 + 0.00084922
[200]	cv_agg's binary_logloss: 0.663889 + 0.00103686
[300]	cv_agg's binary_logloss: 0.662965 + 0.00118076
evaluation:  0.6628777832081384 y_point: -0.002617888821887515
selected: 13029

epoch: 10
parameter:  {'max_depth': 10, 'num_leaves': 63, 'learning_rate': 0.09366421819562454, 'feature_fraction': 0.7063223458068475, 'bagging_fraction': 0.8427090126417702, 'bagging_freq': 2, 'min_data_in_leaf': 25, 'subsample': 0.7721753357568628, 'colsample_bytree': 0.707657072182569, 'reg_lambda'

[100]	cv_agg's binary_logloss: 0.675306 + 0.000440481
[200]	cv_agg's binary_logloss: 0.6698 + 0.000465138
[300]	cv_agg's binary_logloss: 0.666987 + 0.000525712
[400]	cv_agg's binary_logloss: 0.665346 + 0.000701629
[500]	cv_agg's binary_logloss: 0.664362 + 0.000774643
[600]	cv_agg's binary_logloss: 0.663636 + 0.000918259
[700]	cv_agg's binary_logloss: 0.663216 + 0.00101256
[800]	cv_agg's binary_logloss: 0.662947 + 0.0010068
[900]	cv_agg's binary_logloss: 0.662718 + 0.00104707
[1000]	cv_agg's binary_logloss: 0.662673 + 0.00109683
evaluation:  0.6626165077158597 y_point: 0.02350966040598479
selected: 26669

epoch: 19
parameter:  {'max_depth': 3, 'num_leaves': 13, 'learning_rate': 0.08954291705249562, 'feature_fraction': 0.6159209890827922, 'bagging_fraction': 0.8495894709678989, 'bagging_freq': 2, 'min_data_in_leaf': 7, 'subsample': 0.9819362935186499, 'colsample_bytree': 0.6199762770967343, 'reg_lambda': 2.080609506798279, 'reg_alpha': 1.7326649859361116, 'boosting_type': 'gbdt', 'object

[100]	cv_agg's binary_logloss: 0.673162 + 0.000600836
[200]	cv_agg's binary_logloss: 0.66758 + 0.000801318
[300]	cv_agg's binary_logloss: 0.66502 + 0.000977662
[400]	cv_agg's binary_logloss: 0.66389 + 0.00106045
[500]	cv_agg's binary_logloss: 0.663412 + 0.00113512
[600]	cv_agg's binary_logloss: 0.663134 + 0.00120955
[700]	cv_agg's binary_logloss: 0.662994 + 0.00121551
[800]	cv_agg's binary_logloss: 0.662837 + 0.0012535
evaluation:  0.6627201264478071 y_point: 0.013147787211242878
selected: 20554

epoch: 28
parameter:  {'max_depth': 10, 'num_leaves': 10, 'learning_rate': 0.09115474500430822, 'feature_fraction': 0.9583809829456594, 'bagging_fraction': 0.8578581636578525, 'bagging_freq': 1, 'min_data_in_leaf': 4, 'subsample': 0.6826384109052654, 'colsample_bytree': 0.6368540910054228, 'reg_lambda': 14.81744441937647, 'reg_alpha': 14.26300078796009, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 1, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 

parameter:  {'max_depth': 8, 'num_leaves': 16, 'learning_rate': 0.09082326289604241, 'feature_fraction': 0.6948797654915237, 'bagging_fraction': 0.6275737316516378, 'bagging_freq': 2, 'min_data_in_leaf': 24, 'subsample': 0.7564523134768013, 'colsample_bytree': 0.7627866981753254, 'reg_lambda': 0.9889892676372787, 'reg_alpha': 14.291289986528131, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 1, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.669115 + 0.000914774
[200]	cv_agg's binary_logloss: 0.664618 + 0.00125012
[300]	cv_agg's binary_logloss: 0.663115 + 0.00142067
[400]	cv_agg's binary_logloss: 0.66254 + 0.00155587
[500]	cv_agg's binary_logloss: 0.66216 + 0.00165545
evaluation:  0.6621470658229276 y_point: 0.07045384969919244
selected: 26941

epoch: 37
parameter:  {'max_depth': 10, 'num_leaves': 34, 'learning_rate': 0.0924609161223272, 'feature_fraction': 0.9664940360965468, 'bagging_fraction': 0.8604105416371165, 'bagging_freq': 3, 'min_

...

selected: 410

epoch: 55
parameter:  {'max_depth': 9, 'num_leaves': 19, 'learning_rate': 0.09523594061439425, 'feature_fraction': 0.9171610149893128, 'bagging_fraction': 0.7820981064918605, 'bagging_freq': 2, 'min_data_in_leaf': 6, 'subsample': 0.6403095015418124, 'colsample_bytree': 0.6304580217084048, 'reg_lambda': 2.589284933246071, 'reg_alpha': 14.423920908372699, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 1, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.668146 + 0.000729428
[200]	cv_agg's binary_logloss: 0.664073 + 0.00108561
[300]	cv_agg's binary_logloss: 0.662924 + 0.00125894
[400]	cv_agg's binary_logloss: 0.662347 + 0.00130137
[500]	cv_agg's binary_logloss: 0.66209 + 0.0015438
evaluation:  0.6620688395139289 y_point: 0.0782764805990599
selected: 9950

epoch: 56
parameter:  {'max_depth': 10, 'num_leaves': 8, 'learning_rate': 0.0928483818515053, 'feature_fraction': 0.7592726402517473, 'bagging_fraction': 0.717644745398483, 'bagg

[100]	cv_agg's binary_logloss: 0.669802 + 0.000764109
[200]	cv_agg's binary_logloss: 0.665504 + 0.000969858
[300]	cv_agg's binary_logloss: 0.663957 + 0.0010618
[400]	cv_agg's binary_logloss: 0.663398 + 0.00116702
[500]	cv_agg's binary_logloss: 0.663016 + 0.00115537
[600]	cv_agg's binary_logloss: 0.662779 + 0.0011638
[700]	cv_agg's binary_logloss: 0.662603 + 0.00117857
[800]	cv_agg's binary_logloss: 0.662407 + 0.00120481
evaluation:  0.6622805735957314 y_point: 0.05710307241880708
selected: 29904

epoch: 65
parameter:  {'max_depth': 10, 'num_leaves': 28, 'learning_rate': 0.08759837332177983, 'feature_fraction': 0.9516731921161445, 'bagging_fraction': 0.9302655697668922, 'bagging_freq': 1, 'min_data_in_leaf': 22, 'subsample': 0.8508743437399104, 'colsample_bytree': 0.9259147838098662, 'reg_lambda': 14.658186364548703, 'reg_alpha': 14.010929349820444, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 1, 'num_threads': 15}
[100]	cv_agg's binary_loglos

parameter:  {'max_depth': 10, 'num_leaves': 23, 'learning_rate': 0.09154266791032287, 'feature_fraction': 0.6984619357877547, 'bagging_fraction': 0.8844646349901178, 'bagging_freq': 1, 'min_data_in_leaf': 13, 'subsample': 0.8073290597261724, 'colsample_bytree': 0.7959154074620247, 'reg_lambda': 13.219037383329237, 'reg_alpha': 14.994907229026634, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 1, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.668281 + 0.000746642
[200]	cv_agg's binary_logloss: 0.664127 + 0.00103664
[300]	cv_agg's binary_logloss: 0.662977 + 0.000978515
[400]	cv_agg's binary_logloss: 0.662422 + 0.00105788
evaluation:  0.662406392281891 y_point: 0.04452120380284974
selected: 20394

epoch: 74
parameter:  {'max_depth': 9, 'num_leaves': 18, 'learning_rate': 0.08939189748363836, 'feature_fraction': 0.8248073307638999, 'bagging_fraction': 0.956927510114323, 'bagging_freq': 1, 'min_data_in_leaf': 13, 'subsample': 0.9747599969477673,

[300]	cv_agg's binary_logloss: 0.664102 + 0.00107444
[400]	cv_agg's binary_logloss: 0.663226 + 0.00109854
[500]	cv_agg's binary_logloss: 0.662749 + 0.0011647
[600]	cv_agg's binary_logloss: 0.662533 + 0.00125418
[700]	cv_agg's binary_logloss: 0.66234 + 0.00122178
[800]	cv_agg's binary_logloss: 0.662121 + 0.0013607
[900]	cv_agg's binary_logloss: 0.662024 + 0.00137105
evaluation:  0.6619956751629967 y_point: 0.08559291569227856
selected: 20384

epoch: 83
parameter:  {'max_depth': 9, 'num_leaves': 19, 'learning_rate': 0.09131935949831094, 'feature_fraction': 0.736484619410491, 'bagging_fraction': 0.7777074515994122, 'bagging_freq': 1, 'min_data_in_leaf': 3, 'subsample': 0.7138444438878473, 'colsample_bytree': 0.8612020618474329, 'reg_lambda': 6.231254692731381, 'reg_alpha': 14.456147553053116, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 1, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.668718 + 0.000734804
[200]	cv_agg's binary_logloss: 0.6

selected: 22174

epoch: 92
parameter:  {'max_depth': 8, 'num_leaves': 18, 'learning_rate': 0.09076597311143447, 'feature_fraction': 0.9470335226750548, 'bagging_fraction': 0.6446148983022333, 'bagging_freq': 1, 'min_data_in_leaf': 5, 'subsample': 0.7719797843877849, 'colsample_bytree': 0.6864643069254599, 'reg_lambda': 9.178637578121533, 'reg_alpha': 14.96751521926364, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 1, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.669046 + 0.000764727
[200]	cv_agg's binary_logloss: 0.66445 + 0.00113993
[300]	cv_agg's binary_logloss: 0.662967 + 0.00118238
[400]	cv_agg's binary_logloss: 0.662344 + 0.00127479
evaluation:  0.662240629999984 y_point: 0.06109743199355089
selected: 15754

epoch: 93
parameter:  {'max_depth': 9, 'num_leaves': 9, 'learning_rate': 0.0945686708809246, 'feature_fraction': 0.7779777768161134, 'bagging_fraction': 0.708141143731811, 'bagging_freq': 2, 'min_data_in_leaf': 17, 'subsample': 

In [8]:
opt_parameter

{'max_depth': 10,
 'num_leaves': 22,
 'learning_rate': 0.09072376684265862,
 'feature_fraction': 0.6487055938075404,
 'bagging_fraction': 0.735706248678686,
 'bagging_freq': 4,
 'min_data_in_leaf': 18,
 'subsample': 0.6548619211406066,
 'colsample_bytree': 0.9765660260096465,
 'reg_lambda': 14.485624132492505,
 'reg_alpha': 14.874192771862266}

In [9]:
parameters = {**basic_params, **opt_parameter}

In [22]:
parameters["verbose"] = 0

In [52]:
for i in range(5):
    print ("ensamble ", i)
    X_train2, X_test2, y_train2, y_test2 = train_test_split(
                train.values, target_tr.values, test_size=0.1)

    lgb_train = lgb.Dataset(X_train2, y_train2.reshape(-1,))
    lgb_eval = lgb.Dataset(X_test2, y_test2.reshape(-1,), reference=lgb_train)
    model = lgb.train(parameters, lgb_train, valid_sets=lgb_eval, num_boost_round=1000, early_stopping_rounds=10, verbose_eval=False)
    if i == 0:
        result = model.predict(test.values, num_iteration=model.best_iteration)
    else:
        result += model.predict(test.values, num_iteration=model.best_iteration)
result /= 5

ensamble  0
ensamble  1
ensamble  2
ensamble  3
ensamble  4


In [53]:
result_c = (2*result).astype(np.int32) # 確率出力のため, 2倍して1未満かどうかでラベルに変更

In [54]:
print ("正答率：", np.sum(target_te.values == result_c), "/", len(target_te), " ", np.sum(target_te.values == result_c)/len(target_te))

正答率： 6164 / 10294   0.5987954148047406


In [55]:
from sklearn.metrics import accuracy_score
print ("Accuracy:",accuracy_score(target_te.values, result_c))

Accuracy: 0.5987954148047406
